In [1]:
# Develop features for demo, etc. 
# But, first step is to create DF from Recommendation
import collections
import compress_pickle
import itertools
import libsbml
import numpy as np
import os
import pickle
import pandas as pd
import sys
import time
import matplotlib.pyplot as plt
%matplotlib inline  

BIOMD_12 = 'BIOMD0000000012.xml'
BASE_DIR = '/Users/woosubs/Desktop/AutomateAnnotation/'
DATA_DIR = os.path.join(BASE_DIR, "DATA")
ALGO_DIR = os.path.join(DATA_DIR, "algo")
CHEBI_DIR = os.path.join(DATA_DIR, "chebi")
RHEA_DIR = os.path.join(DATA_DIR, "rhea")
BIOMODEL_DIR = os.path.join(DATA_DIR, "biomodels/curated_biomodels_31mar2021")
BIGG_DIR = '/Users/woosubs/Desktop/AutomateAnnotation/DATA/bigg'
ecoli_fpath = os.path.join(BIGG_DIR, "e_coli_core.xml")

PROJ_DIR = os.path.join(os.getcwd(), os.pardir)
AMAS_DIR = os.path.join(PROJ_DIR, "AMAS")
sys.path.append(PROJ_DIR)

SUPPL_DIR = os.path.join(PROJ_DIR, os.pardir, "AMAS_suppl")
ACCURACY_DIR = os.path.join(SUPPL_DIR, "data_for_credibility")
RESULT_DIR = '/Users/woosubs/Desktop/AutomateAnnotation/AMAS_suppl/result_files'

from AMAS import annotation_maker as am
from AMAS import species_annotation as sa
from AMAS import reaction_annotation as ra
from AMAS import recommender
from AMAS import constants as cn
from AMAS import iterator as it
from AMAS import tools

In [2]:
E_COLI_PATH = '/Users/woosubs/Desktop/AutomateAnnotation/AMAS/AMAS/../tests/e_coli_core.xml'
BIOMD_190_PATH = os.path.join(cn.TEST_DIR, 'BIOMD0000000190.xml')

In [3]:
recom = recommender.Recommender(libsbml_fpath=BIOMD_190_PATH)

In [73]:
recom.getSpeciesIDs()

['SAM', 'A', 'P', 'S', 'D', 'aS', 'aD', 'Met', 'ORN', 'AcCoA', 'CoA']

In [108]:
command = "python recommend_species.py --model files/BIOMD0000000190.xml "
command += "--species SAM A --min_score 0.6 --out_dir res"
command

'python recommend_species.py --model files/BIOMD0000000190.xml --species SAM A --min_score 0.6 --out_dir res'

In [6]:
# TODO: developing 'recommendSpecies/Reaction' as a command line tool
# (1) Need to specify the workflow (make sure which arguments are needed)
# (2) and implement the rest 

In [7]:
# for reaction

##########################
# arguments :) 
one_fpath = BIOMD_190_PATH
ids2pred = ['ODC', 'SAMdc']
min_threshold = 0.6

recom = recommender.Recommender(libsbml_fpath=one_fpath)
recom.current_type = 'reaction'
res = recom.getReactionListRecommendation(ids2pred, get_df=True)
for idx, one_df in enumerate(res):
  filt_df = recom.autoSelectAnnotation(one_df, min_threshold)
  # (1) create dataframe/table here instead of storing
  # (2) update self.selection based on this
  recom.updateSelection(ids2pred[idx], filt_df)
# store file to csv
recom.saveToSBML()
print("Done with ... IDs updated etc.")

Annotation of ODC saved.
Done with ... IDs updated etc.


In [8]:
recom.displaySelection()

                                        ODC (credibility score: 0.817)                                       
+----+--------------+---------------+-----------------------------------------------------------------------+
|    | annotation   |   match score | label                                                                 |
+====+==============+===============+=======================================================================+
|  1 | RHEA:28827   |         1.000 | L-ornithine(out) + putrescine(in) = L-ornithine(in) + putrescine(out) |
+----+--------------+---------------+-----------------------------------------------------------------------+

     SAMdc (credibility score: 0.587)     
+--------------+---------------+---------+
| annotation   | match score   | label   |
+==============+===============+=========+
+--------------+---------------+---------+



In [12]:
recom.saveToCSV()

In [11]:
# make selection to csv saveToCSV
# def saveToCSV(fpath="annotation_table.csv"):
#   """
#   Save self.selection to csv.
#   """
#   dfs = []
#   pd.set_option('display.max_colwidth', 255)
#   for one_type in recom.selection.keys():
#     type_selection = recom.selection[one_type]
#     if any(type_selection):
#       element_df = pd.concat([type_selection[k] for k in type_selection.keys()])
#       element_ids = list(itertools.chain(*[[k]*type_selection[k].shape[0] \
#                                            for k in type_selection.keys()]))
#       element_types = [one_type]*len(element_ids)
#       element_df.insert(0, "type", element_types)
#       element_df.insert(1, "id", element_ids)
#       dfs.append(element_df)
#   res = pd.concat(dfs)
#   # Write result to csv
#   res.to_csv(fpath, index=False) 

In [54]:
print(element_df['label'])

1    L-ornithine(out) + putrescine(in) = L-ornithine(in) + putrescine(out)
Name: label, dtype: object


In [9]:
# finally, check if the file has been successfully updated
reader = libsbml.SBMLReader()
document = reader.readSBML(os.path.join(os.getcwd(), 'updated_model.xml'))
model = document.getModel()
print(model.getReaction('ODC').getAnnotationString())

<annotation>
  <rdf:RDF xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#" xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:dcterms="http://purl.org/dc/terms/" xmlns:vCard="http://www.w3.org/2001/vcard-rdf/3.0#" xmlns:bqbiol="http://biomodels.net/biology-qualifiers/" xmlns:bqmodel="http://biomodels.net/model-qualifiers/">
    <rdf:Description rdf:about="#metaid_0000062">
      <bqbiol:is>
        <rdf:Bag>
          <rdf:li rdf:resource="http://identifiers.org/rhea/RHEA:28827"/>
        </rdf:Bag>
      </bqbiol:is>
    </rdf:Description>
  </rdf:RDF>
</annotation>


In [13]:
recom.getSpeciesRecommendation(pred_id='SAM', get_df=True)

,annotation,match score,label
SAM (cred. 0.794),,,
1,CHEBI:67040,1.0,S-adenosyl-L-methioninate
2,CHEBI:15414,1.0,S-adenosyl-L-methionine


In [83]:
# finally, check if the file has been successfully updated
reader = libsbml.SBMLReader()
document = reader.readSBML(os.path.join(os.getcwd(), 'updated_model.xml'))
model = document.getModel()
print(model.getReaction('SAMdc').getAnnotationString())

In [ ]:
if cands2save:
  print("some given")

In [ ]:
df = res[1]
df

,annotation,match score,label
SAMdc (cred. 0.587),,,
1,RHEA:15981,0.5,adenosylmethionine decarboxylase activity


In [ ]:
autoSelectAnnotation(df, 0.2)

,annotation,match score,label
SAMdc (cred. 0.587),,,
1,RHEA:15981,0.5,adenosylmethionine decarboxylase activity


In [ ]:
autoSelectAnnotation

In [ ]:
empty_df = recom.filterDataFrameByThreshold(res[1], min_threshold)
empty_df.shape

(0, 3)

In [8]:
# def autoSelectAnnotation(df, min_score=0.0):
#   """
#   Choose annotations based on 
#   the set threshold; 
#   (1) if None meets the threshold, return an empty frame
#   (2) if multiple meet the threshold,
#       (a) find the highest match score among them
#       (b) return all above match score == highest_match_score
      
#   Parameters
#   ----------
#   df: pandas.DataFrame
  
#   min_score: float (0.0-1.0)
  
#   Returns
#   -------
#   pandas.DataFrame
#     if nothing matched,
#     an empty dataframe is returned
#   """ 
#   scores = df['match score']
#   # max_score: highest match score that exists
#   # min_score: threshold
#   max_score = np.max(scores)
#   if max_score < min_score:
#     # this will create an empty dataframe
#     filt_idx = scores[scores>=min_score].index
#   else:
#     filt_idx = scores[scores==max_score].index
#   filt_df = df.loc[filt_idx, :]  
#   return filt_df

<annotation>
  <rdf:RDF xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#" xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:dcterms="http://purl.org/dc/terms/" xmlns:vCard="http://www.w3.org/2001/vcard-rdf/3.0#" xmlns:bqbiol="http://biomodels.net/biology-qualifiers/" xmlns:bqmodel="http://biomodels.net/model-qualifiers/">
    <rdf:Description rdf:about="#metaid_0000062">
      <bqbiol:is>
        <rdf:Bag>
          <rdf:li rdf:resource="http://identifiers.org/rhea/RHEA:28827"/>
        </rdf:Bag>
      </bqbiol:is>
    </rdf:Description>
  </rdf:RDF>
</annotation>


In [ ]:
# TODO:


In [97]:
import argparse

In [114]:
parser = argparse.ArgumentParser(description='SBML file (.XML) and one or more species IDs in the model.')

In [115]:
parser.add_argument('--model', type=str, help='SBML file in the XML format')
parser.add_argument('--species', type=str, help='ID of species in the model', nargs="*")
parser.add_argument('--min_score', type=float, help='Minimum threshold of recommendation')
parser.add_argument('--outfile_dir', type=str, help='Path of directory to save files')

_StoreAction(option_strings=['--outfile_dir'], dest='outfile_dir', nargs=None, const=None, default=None, type=<class 'str'>, choices=None, required=False, help='Path of directory to save files', metavar=None)

In [116]:
args = parser.parse_args(['--model', 'updated_model.xml',
                          '--min_score', '0.6',
                          '--outfile_dir', 'recommended'])

In [117]:
args.species

In [146]:
# def printSummary(saved_elements, element_type):
#   """
#   Print out a summary of 
#   saved element IDs and numbers. 
  
#   Parameters
#   ----------
#   saved_elements: list-str
#       List of elements saved. 
#   """
#   plural_str = {'species': ' were',
#                 'reaction': 's were'}
#   num_saved = len(saved_elements)
#   if num_saved == 0:
#     return None
#   str2add = ""
#   if num_saved==1:
#      print("Recommendation of %s %s was saved.\n" % \
#            (element_type, saved_elements[0]))
#   elif num_saved>1:
#     print("Recommendations of %d %s%s saved:\n[%s]\n" %\
#           (num_saved,
#            element_type,
#            plural_str[element_type],
#            ', '.join(saved_elements))) 

In [147]:
printSummary(['SAM', 'A'], 'species')

Recommendations of 2 species were saved:
[SAM, A]



In [148]:
printSummary(['SAM'], 'species')

Recommendation of species SAM was saved.



In [149]:
printSummary(['ODC', 'SAMdc'], 'reaction')

Recommendations of 2 reactions were saved:
[ODC, SAMdc]



In [150]:
printSummary(['ODC'], 'reaction')

Recommendation of reaction ODC was saved.



In [152]:
E_COLI_PATH.split('/')[-1].split('.')[0]

'e_coli_core'